In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("1-1.여성의류(196).json")
df.head()

## 문제 
- 데이터프레임에서 'Aspects' 컬럼에 데이터들을 이용하여 분류 모델을 생성하려 한다. 
- SentimentText 텍스트를 이용하여 'Aspect', 'SentimentPolarity'의 값들을 예측 하는 모델을 생성 
    1. df에서 'Aspects' 데이터를 추출 
    2. SentimentText데이터를 문자형으로 이루어져있으니 학습에 대한 데이터의 형태로 변환 (문자의 데이터를 숫자형 데이터) -> 토큰화(Okt), 벡터화(TF-IDF)
    3. 종속 변수는 'Aspect', 'SentimentPolarity'
    4. 분류 모델(LinearSVC) random_state만 42로 고정 
    5. 테스트를 이용하여 분류가 잘되고 있는가? 정확도만 확인(197 데이터를 로드하여 정확도 계산)
    - 백터화, 모델링 파이프라인으로 연결해서 사용

In [ ]:
import json

In [ ]:
json.load(
    open('1-1.여성의류(196).json', 'r', encoding='utf-8')
)

In [ ]:
test_text = [
    '색상이 마음에 든다', 
    '설명에 비해 옷이 두껍진 않다', 
    '길이가 너무 길지도 않고 짧지도 한다.'
]

In [ ]:
# 라이브러리 로드 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [ ]:
# 데이터 로드 
df = pd.read_json("1-1.여성의류(196).json")
df.head(1)

In [ ]:
# Aspects 컬럼의 데이터들을 이용하여 하나의 새로운 데이터프레임으로 생성 
pd.DataFrame(df['Aspects'].sum())

In [ ]:
# 각 행의 Aspects 데이터를 추출하여 리스트에 추가 
new_list= []

# iterrows() : 데이터프레임에서 [(idx, df.loc[idx, ]), ...] 되돌려주는 
for i, s in df.iterrows():
    # print(s['Aspects'])
    aspects = s['Aspects']
    for aspect in aspects:
        # aspect에서 'SentimentWord' 키는 제거 
        # dict 형태의 데이터에서 특정 키를 제거 -> del
        try:
            del aspect['SentimentWord']
        except:
            pass
        # print(aspect)
        new_list.append(aspect)

aspect_df = pd.DataFrame(new_list)
        

In [ ]:
# 분류 모델을 돌리기 위해 종속 변수들의 균형이 어떻게 되는가?
# polarity의 데이터의 개수들을 확인 
aspect_df['SentimentPolarity'].value_counts()

In [ ]:
aspect_df['Aspect'].value_counts()

In [ ]:
# 결측치가 존재하는가?
aspect_df.isna().sum()

In [ ]:
# 모든 value에게 좌우의 공백을 제거
aspect_df.iloc[:, :2] = aspect_df.iloc[:, :2].map(
    lambda x : x.strip()
)

In [ ]:
# 빈텍스트가 존재하는가?
aspect_df.isin(['']).sum()

In [ ]:
# Aspect 컬럼의 데이터들을 LabelEncoder로 변환 
le = LabelEncoder()

aspect_df['Aspect'] = le.fit_transform(aspect_df['Aspect'])

In [ ]:
aspect_df.head()

In [ ]:
# Text을 토큰화 -> 백터화 작업
from konlpy.tag import Okt

In [ ]:
okt = Okt()

vectorizer = TfidfVectorizer(
    tokenizer= okt.morphs, 
    lowercase= False, 
    ngram_range=(1, 2)
)

In [ ]:
# 분류 모델 정의 
svc1 = LinearSVC(
    random_state= 42
)
svc2 = LinearSVC(
    random_state= 42
)

In [ ]:
# Aspect 예측하기 위한 모델 
pipe_aspect = Pipeline(
    [
        ('vector', vectorizer), 
        ('clf', svc1)
    ]
)
# Pola 예측하기 위한 모델 
pipe_pola = Pipeline(
    [
        ('vector', vectorizer), 
        ('clf', svc2)
    ]
)

In [ ]:
X = aspect_df['SentimentText'].values
Y1 = aspect_df['Aspect'].values
Y2 = aspect_df['SentimentPolarity'].values

In [ ]:
Y2

In [ ]:
# 2개의 모델에 학습 -> 독립 변수 Text, 종속 변수 Aspect, Pola
# pipeline.fit() -> 스탭중 변환이 있으면 fit_transform() -> 모델에는 fit()
pipe_aspect.fit(X, Y1)
pipe_pola.fit(X, Y2)

In [ ]:
# pipeline.predit() -> 스탭 중 변환이 있으면 transform() -> 모델에는 predict()
pred_aspect = pipe_aspect.predict(test_text)
pred_pola = pipe_pola.predict(test_text)

In [ ]:
pred_pola

In [ ]:
pred_aspect

In [ ]:
le.inverse_transform(pred_aspect)

In [ ]:
test_text

In [ ]:

df2 = aspect_df.copy()

In [ ]:
# LabelEncoder를 원본으로 변환 
df2['Aspect'] = le.inverse_transform(df2['Aspect'])

In [ ]:
df2['target'] = df2['Aspect'] + '_' + df2['SentimentPolarity']

In [ ]:
df2.head()

In [ ]:
# target 컬럼의 데이터를 LabelEncoder 변환 
le2 = LabelEncoder()
df2['target'] = le2.fit_transform(df2['target'])

In [ ]:
X_2 = df2['SentimentText']
Y_2 = df2['target']

In [ ]:
print(X_2.shape, Y_2.shape)

In [ ]:
pipe = Pipeline(
    [
        ('vector', vectorizer ),
        ('clf', LinearSVC(random_state=42))
    ]
)

In [ ]:
pipe.fit(X_2, Y_2)

In [ ]:
pipe.predict(test_text)

In [ ]:
# 종속 변수가 2개인 경우 일반적으로 사용하는 객체 
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
# 종속 변수의 크기가 (2000, 2)
# 첫번째 종속의 데이터를 이용하여 fit() -> predict()
# 두번째 종속의 데이터를 이용하여 fit() -> predict()
# 위의 2개의 작업를 병렬로 처리 

In [ ]:
# 분류 모델을 생성 
svc = LinearSVC(random_state=42)
# 멀티 아웃 모델을 생성 
multi_model = MultiOutputClassifier(svc)
# 파이프라인 생성 
pipe_multi = Pipeline(
    [
        ('vector', vectorizer), 
        ('model', multi_model)
    ]
)

In [ ]:
aspect_df['SentimentPolarity'] = aspect_df['SentimentPolarity'].astype(int)

In [ ]:
# 멀티 모델 종속은 2차원 그대로 사용 
X = aspect_df['SentimentText'].values
Y1 = aspect_df[['Aspect', 'SentimentPolarity']].values

In [ ]:
Y1

In [ ]:
pipe_multi.fit(X, Y1)

In [ ]:
# 예측 
pred_multi = pipe_multi.predict(test_text)

In [ ]:
pred_aspect_origin = le.inverse_transform(pred_multi[:, 0])
pred_aspect_origin

In [ ]:
pred_pola = pred_multi[:, 1]
pred_pola

In [ ]:
# 문단인 장문의 데이터에서 문장별로 나눠주기 
from konlpy.tag import Kkma

In [ ]:
text = df.loc[2, 'RawText']

In [ ]:
kkma = Kkma()

In [ ]:
texts = kkma.sentences(text)

In [ ]:
pred = pipe_multi.predict(texts)

In [ ]:
le.inverse_transform(pred[:, 0])

In [ ]:
pred[:, 1]

In [ ]:
texts

#### 최적의 파라미터를 확인
- pipe_multi에서 최적의 파라미터를 생성
- step 1
    - vector화 작업
        - ngram_range를 (1, 1), (1, 2)
- step2
    - LinearSVC를 병렬 작업
        - C 를 1.0, 1.5, 2.0
        - class_weight 를 None, 'balanced'
- 교차 검증의 횟수는 3회 
- 평가 점수는 정확도
- 최적의 매개변수의 값을 확인 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_params = {
    'vector__ngram_range' : [ (1, 1), (1, 2) ], 
    'model__estimator__C' : [ 1.0, 1.5, 2.0 ], 
    'model__estimator__class_weight' : [ None, 'balanced' ]
}

In [ ]:
grid = GridSearchCV(
    estimator= pipe_multi, 
    param_grid= grid_params, 
    cv = 3, 
    scoring='accuracy', 
    n_jobs=1
)

In [ ]:
grid.fit(X, Y1)

In [ ]:
grid.best_score_